In [8]:
import os
import sys
import time
import sqlite3
import multiprocessing
sys.path.append('/home/dnlab/Jupyter-Bitcoin/Heuristics/ExperimentSpeed/')
sys.path.append('/home/dnlab/Jupyter-Bitcoin/Heuristics')

from secret import rpc_user, rpc_password
from bitcoinrpc.authproxy import AuthServiceProxy, JSONRPCException

import test_cluster_db_query as cdq
import test_db_query as dq

rpc_ip = '127.0.0.1'
rpc_port = '8332'
timeout = 300

In [9]:
def is_mi_cond(in_addrs, out_addrs):
    if in_addrs == None or out_addrs == None:
        return False
    if len(out_addrs) == 1 and len(in_addrs) >= 2:
        return True
    return False

In [10]:
def get_rpc():
    return AuthServiceProxy(f'http://{rpc_user}:{rpc_password}@{rpc_ip}:{rpc_port}', timeout=timeout)

def rpc_command(height):
    while True:
        try:
            rpc_connection = get_rpc()
            block_hash = rpc_connection.getblockhash(height)
            txes = rpc_connection.getblock(block_hash)['tx']
            break
        except OSError as e:
            print("Cannot assign requested address!")
            time.sleep(3)
    return txes

In [11]:
cluster_db_path = '/home/dnlab/Jupyter-Bitcoin/Heuristics/DB/cluster_10_multitest.db '
index_db_path = '/home/dnlab/Jupyter-Bitcoin/index.db'
i_conn = sqlite3.connect(index_db_path)
c_conn = sqlite3.connect(cluster_db_path)
icur = i_conn.cursor()
ccur = c_conn.cursor()

In [12]:
# 가변길이
height = 100000

In [13]:
false_list = list()
tx_count = 0
mi_tx = 0
address_count = 0
for height in range(1, dq.get_max_height()+1):
    txes = rpc_command(height)
    #print("tx length:", len(txes))
    for tx in txes:
        tx_count += 1

        tx_indexes = dq.get_txid(tx)
        in_addrs = dq.get_addr_txin(tx_indexes)
        out_addrs = dq.get_addr_txout(tx_indexes)
        address_count += 1
        #print(f'tx: {tx_indexes}, in_addrs:{in_addrs}, out_addrs:{out_addrs}')
        if is_mi_cond(in_addrs, out_addrs):
            mi_tx += 1
            #print(f'tx: {tx_indexes}, in_addrs:{in_addrs}, out_addrs:{out_addrs}')
            cluster_set = cdq.get_cluster_number(in_addrs)
            if len(cluster_set) > 1:
                print("[TEST CASE FALSE] different cluster_number", tx, in_addrs, len(out_addrs), cluster_set)
                false_case = {'false_num':0, 'tx':tx, 'in_addrs': in_addrs, \
                              'out_addrs':out_addrs, 'cluster_set':cluster_set}
                false_list.append(false_case)
            elif cluster_set.pop() == -1:
                print("[TEST CASE FALSE] not clustered tx:", tx,", in_addrs: ", len(in_addrs), len(out_addrs), ", cluster_set", '{-1}')
                false_case = {'false_num':1, 'tx':tx, 'in_addrs': in_addrs, \
                              'out_addrs':out_addrs, 'cluster_set':{-1}}
                print(in_addrs)
                false_list.append(false_case)
            else:
                #print("*TEST CASE PASS*")
                pass
    if height % 100000 == 0:
        print("height ", height, "PASS all tx: ", tx_count, "mi_tx: ", mi_tx)
print("height ", height, "PASS all tx: ", tx_count, "mi_tx: ", mi_tx)

height  100000 PASS all tx:  216576 mi_tx:  6730
height  200000 PASS all tx:  7316695 mi_tx:  118629
height  300000 PASS all tx:  38463788 mi_tx:  565428
height  400000 PASS all tx:  112477765 mi_tx:  2094734
height  500000 PASS all tx:  283595038 mi_tx:  6377554
height  600000 PASS all tx:  466297404 mi_tx:  12245027
[TEST CASE FALSE] different cluster_number 90dc1e5ba8b79566288775e3e7189a763705093589bc4ceaf32dad58d7a1d353 {'628871218', '628871217', '620549013'} 1 {620549013, -1}
[TEST CASE FALSE] different cluster_number a88f9f17a4d5146f2ff7584b6de436885ce8f747bdc16799fe433f788bce53c7 {'620736773', '628870703'} 1 {620736773, -1}
[TEST CASE FALSE] different cluster_number eeb5f8d19c56496422e65f87bae5ecf206be9177d3c8f3936eec7e445f48f0ea {'628871337', '574028051'} 1 {574028051, -1}
[TEST CASE FALSE] different cluster_number 895475fc4c492b0f2bbb34a6c19c0b8d7b1282d529bed313849e30eca028d7ef {'614375721', '614409036', '615504139', '628871400', '627610427'} 1 {614375721, -1}
[TEST CASE FALSE

In [14]:
import csv
with open('/home/dnlab/Jupyter-Bitcoin/Heuristics/debug/20070029.csv', 'a') as f:
    fieldnames = ['false_num', 'tx', 'in_addrs','out_addrs','cluster_set']
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    for i in false_list:
        writer.writerow(i)     

In [15]:
tx_count#216573
print(mi_tx)
print(tx_count)

13816249
508926291
